In [1]:
import yfinance as yf

df= yf.download("TCS.NS", start="2021-07-01", end="2022-01-28")
import pandas as pd
if isinstance(df.columns, pd.MultiIndex):
    df.columns = df.columns.get_level_values(0)
df.columns.name = None 
dataset=df.drop(columns=['Volume'])
dataset

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


,Close,High,Low,Open
Date,,,,
2021-07-01,3117.359619,3136.484525,3108.823472,3132.752836
2021-07-02,3102.385986,3115.680127,3074.165091,3107.563750
2021-07-05,3098.234131,3147.679000,3080.508612,3117.825494
2021-07-06,3043.472412,3104.765361,3037.688340,3090.771526
2021-07-07,3055.273193,3062.503340,3024.160378,3055.319885
...,...,...,...,...
2022-01-20,3590.827637,3678.520902,3576.235499,3669.136920
2022-01-21,3597.349365,3614.287498,3538.793411,3572.481814
2022-01-24,3539.028564,3612.504969,3509.703617,3603.449517


In [2]:
#column='Close'

In [2]:
from sklearn.preprocessing import MinMaxScaler
Ms=MinMaxScaler()
data1=Ms.fit_transform(dataset)
print('Len:',data1.shape)

Len: (144, 4)


In [3]:
len(data1)

144

In [4]:
training_size=round(len(data1)* 0.85)
print(training_size)
x_train=data1[:training_size]
x_test=data1[training_size:]
print("x_train length:",x_train.shape)
print("x_test length:",x_test.shape)
y_train=data1[:training_size]
y_test=data1[training_size:]
print("x_train length:",y_train.shape)
print("x_test length:",y_test.shape)

122
x_train length: (122, 4)
x_test length: (22, 4)
x_train length: (122, 4)
x_test length: (22, 4)


In [27]:
import warnings
warnings.filterwarnings("ignore")

In [6]:
data1=pd.DataFrame(data1,columns=["Open","High","Low","Close"])

In [20]:
from statsmodels.tsa.statespace.varmax import VARMAX
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
import pandas as pd
import numpy as np

# Define global performance tracking dictionary
performance = {
    "Model": [],
    "RMSE": [],
    "MaPe": [],
    "Lag": [],
    "Test": []
}

def cominbation(dataset, listt):
    print("Selected columns:", listt)
    datasetTwo = dataset[listt]

    test_obs = 28
    train = datasetTwo[:-test_obs]
    test = datasetTwo[-test_obs:]

    best_aic = np.inf
    best_order = None
    best_model = None

    # Search over VARMA(p, q) combinations
    for p in range(0, 4):  # AR order
        for q in range(0, 4):  # MA order
            if p == 0 and q == 0:
                continue  # skip (0,0)
            try:
                model = VARMAX(train, order=(p, q), enforce_stationarity=True)
                results = model.fit(disp=False)
                print(f"Order = ({p},{q}), AIC = {results.aic}")

                if results.aic < best_aic:
                    best_aic = results.aic
                    best_order = (p, q)
                    best_model = results
            except Exception as e:
                print(f"Failed for order=({p},{q}): {e}")
                continue

    print(f"\n✅ Best order: {best_order} with AIC: {best_aic}")

    # Forecast using the best model
    pred = best_model.forecast(steps=test_obs)
    preds = pd.DataFrame(pred, columns=listt)

    # Save predictions
    preds.to_csv(f"varma_forecasted_{test_obs}.csv", index=False)

    # Evaluate forecast
    rmse = round(mean_squared_error(test, preds, squared=False))
    mape = mean_absolute_percentage_error(test, preds)

    # Log performance
    performance["Model"].append(listt)
    performance["RMSE"].append(rmse)
    performance["MaPe"].append(mape)
    performance["Lag"].append(best_order)
    performance["Test"].append(test_obs)

    perf = pd.DataFrame(performance)
    return perf, best_model, preds


In [24]:
listt=["Open","High","Low","Close"]

In [25]:
perf,result,pred=cominbation(data1,listt)

Selected columns: ['Open', 'High', 'Low', 'Close']


C:\Anaconda\envs\aiml\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Order = (0,1), AIC = -1617.2256153630108


C:\Anaconda\envs\aiml\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Order = (0,2), AIC = -1582.436394707427


...

Order = (3,1), AIC = -2005.0472674295847


C:\Anaconda\envs\aiml\Lib\site-packages\statsmodels\tsa\statespace\varmax.py:160: EstimationWarning: Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.
  warn('Estimation of VARMA(p,q) models is not generically robust,'


Order = (3,2), AIC = -1976.6425489354583


C:\Anaconda\envs\aiml\Lib\site-packages\statsmodels\tsa\statespace\varmax.py:160: EstimationWarning: Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.
  warn('Estimation of VARMA(p,q) models is not generically robust,'
C:\Anaconda\envs\aiml\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Order = (3,3), AIC = -1958.8811567484709

✅ Best order: (1, 0) with AIC: -2059.416654586331


In [26]:
perf

,Model,RMSE,MaPe,Lag,Test
0,"[Close, High]",0,0.321843,"(2, 0)",28
1,"[Open, High, Low, Close]",0,0.330332,"(1, 0)",28
